In [1]:
import torch
import torch.nn as nn

In [3]:
#单向单层
single_rnn = nn.RNN(4, 3, 1, batch_first=True)
input=torch.randn(1,2,4)
output, h_n = single_rnn(input)
print(output)

tensor([[[ 0.3296, -0.5430, -0.2576],
         [ 0.7505, -0.6712, -0.6390]]], grad_fn=<TransposeBackward1>)


In [7]:
##双向单层
bi_rnn = nn.RNN(4, 3, 1, batch_first=True,bidirectional=True)
output, bi_h_n = bi_rnn(input)
print(output.shape)
print(bi_h_n.shape)
print(output)

torch.Size([1, 2, 6])
torch.Size([2, 1, 3])
tensor([[[ 0.1598, -0.6498, -0.7726,  0.4034, -0.2161, -0.2369],
         [-0.4590, -0.5766, -0.9310,  0.0950, -0.1516, -0.1073]]],
       grad_fn=<TransposeBackward1>)


In [12]:
bs, T = 2, 3 #batch_size,Seq_len
input_size, hidden_size = 2, 3 ##输入特征向量长度，隐层大小
input = torch.randn(bs, T, input_size)
h_prev = torch.zeros(bs, hidden_size) ##t0
##调用RNN
rnn = nn.RNN(input_size, hidden_size, batch_first=True)
rnn_output, state_final = rnn(input, h_prev.unsqueeze(0))
print(rnn_output)
print(state_final)

tensor([[[-0.2624, -0.0930,  0.5239],
         [-0.5348,  0.0868,  0.4009],
         [-0.9487, -0.6711, -0.6736]],

        [[-0.2104,  0.0113,  0.3902],
         [-0.6886, -0.2108,  0.2436],
         [-0.9304, -0.5874, -0.7124]]], grad_fn=<TransposeBackward1>)
tensor([[[-0.9487, -0.6711, -0.6736],
         [-0.9304, -0.5874, -0.7124]]], grad_fn=<StackBackward0>)


In [18]:
def rnn_forward(input, weight_ih, weight_hh, bias_ih, bias_hh, h_prev):
    bs, T, input_size = input.shape
    h_dim = weight_ih.shape[0]
    h_out = torch.zeros(bs, T, h_dim)
    
    for t in range(T):
        x = input[:, t ,:].unsqueeze(2) ##batch_size,t,input_size
        w_ih_batch = weight_ih.unsqueeze(0).tile(bs, 1, 1) ##bs*h_dim*is
        w_hh_batch = weight_hh.unsqueeze(0).tile(bs, 1, 1) #bs*h_dim*h_dim
        w_times_x = torch.bmm(w_ih_batch, x).squeeze(-1) ##bs*h_dim
        w_times_h = torch.bmm(w_hh_batch, h_prev.unsqueeze(2)).squeeze(-1)
        h_prev = torch.tanh(w_times_x + bias_ih + w_times_h + bias_hh)
        h_out [:,t,:] = h_prev
    return h_out, h_prev.unsqueeze(0)

for k, v in rnn.named_parameters():
    print(k, v)

weight_ih_l0 Parameter containing:
tensor([[0.4624, 0.4869],
        [0.3447, 0.2113],
        [0.1223, 0.5683]], requires_grad=True)
weight_hh_l0 Parameter containing:
tensor([[ 0.3348,  0.4235, -0.4712],
        [ 0.4829, -0.4085,  0.4250],
        [ 0.4649,  0.5510,  0.0752]], requires_grad=True)
bias_ih_l0 Parameter containing:
tensor([-0.3637, -0.3360,  0.4538], requires_grad=True)
bias_hh_l0 Parameter containing:
tensor([-0.1998,  0.1260, -0.2486], requires_grad=True)


In [19]:
rnn_forward(input, rnn.weight_ih_l0, rnn.weight_hh_l0, rnn.bias_ih_l0, rnn.bias_hh_l0, h_prev)

(tensor([[[-0.2624, -0.0930,  0.5239],
          [-0.5348,  0.0868,  0.4009],
          [-0.9487, -0.6711, -0.6736]],
 
         [[-0.2104,  0.0113,  0.3902],
          [-0.6886, -0.2108,  0.2436],
          [-0.9304, -0.5874, -0.7124]]], grad_fn=<CopySlices>),
 tensor([[[-0.9487, -0.6711, -0.6736],
          [-0.9304, -0.5874, -0.7124]]], grad_fn=<UnsqueezeBackward0>))